In [ ]:
import re

In [ ]:
def p_groups(m):
    print m.group()
    for ix, g in enumerate(m.groups()):
        print g, m.start(ix+1), m.end(ix+1)

The regex matching algorithm is
* greedy
* non-overlapping

## Greedy

### string search

the search is not greedy.

In [ ]:
"first blue second blue".find("blue")

In [ ]:
single_word = re.compile(r'blue')

In [ ]:
m = single_word.search("first blue second blue")
m.start(), m.end()

### add repetition qualifiers to the regex

two groups:
* in the first group is the last letter followed by at least one digit (separated by any character)
* second group the last digit when there was at least one letter in front

important are the two **.\*** in the regex.

In [ ]:
letters_digits = re.compile(r'.*([a-z]+).*([0-9]+)')

In [ ]:
letters_digits.findall("__abc__123__d")

In [ ]:
letters_digits.match("__abc__123__d").groups()

change the __.\*__ to expected delimiters **_+**

two groups:
* letters followed by at least one digit, optionally separated by delimiter
* digits preceeded by at least one letter, optionally separated by delimiter

In [ ]:
letters_digits_delimiter = re.compile(r'[_+]*([a-z]+)[_+]*([0-9]+)')

In [ ]:
letters_digits_delimiter.findall("__abc__123__d")

In [ ]:
letters_digits_delimiter.findall("__abc__123__456__de__78")

In [ ]:
letters_digits_delimiter.findall("__abc123de45")

In [ ]:
for m in letters_digits_delimiter.finditer("__abc123de45"):
    p_groups(m)

## Overlapping Matches

regex find all non-overlapping matches

In [ ]:
only_a = re.compile(r'aa')

In [ ]:
only_a.findall("aaa")

In [ ]:
for m in only_a.finditer("aaa"):
    print m.group()
    print (m.start(), m.end())

In [ ]:
m = only_a.search("aaa")
m.start(), m.end()

which strings qualify for overlapping matches?
* string of a single character
* string repeating its start at the end

### match string of a single letter

* "aaaaa" -> is a match
* "aabaa" -> not a match

In [ ]:
one = re.compile(r'^([a-z])(\1+)$')

In [ ]:
one.match("aaaaa").groups()

In [ ]:
one.findall("aabaa")

### match string repeating its start at the end
"abab"
"aacaa"
"aa"
"ababab"

In [ ]:
start_repeats_at_end = re.compile(r'([a-z]+).*(\1+)')

In [ ]:
for m in start_repeats_at_end.finditer("abababccab"):
    p_groups(m)

In [ ]:
for m in start_repeats_at_end.finditer("ababab"):
    p_groups(m)